In [1]:
import pandas as pd

In [2]:
file = "raw/Planner_format.csv"
Asana_format = file[4:-4]+"clean.csv"

In [3]:
df = pd.read_csv(file)

In [4]:
df.sort_values("Bucket Name", ignore_index=True, inplace=True)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77 entries, 0 to 76
Data columns (total 18 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   Task ID                    77 non-null     object
 1   Task Name                  77 non-null     object
 2   Bucket Name                77 non-null     object
 3   Progress                   77 non-null     object
 4   Priority                   77 non-null     object
 5   Assigned To                32 non-null     object
 6   Created By                 77 non-null     object
 7   Created Date               77 non-null     object
 8   Start Date                 36 non-null     object
 9   Due Date                   34 non-null     object
 10  Is Recurring               77 non-null     bool  
 11  Late                       77 non-null     bool  
 12  Completed Date             1 non-null      object
 13  Completed By               1 non-null      object
 14  Description 

In [6]:
df.insert(0, "Order3", range(0, len(df)*2, 2))

In [7]:
df["Progress"] = df["Progress"].replace("Completed", "Done")

In [8]:
df.rename(columns = {"Task Name":"Name", "Bucket Name":"Section/Column", "Assigned To": "Collaborators",
"Created Date": "Created At", "Completed Date": "Completed At", "Description":"Notes", "Progress":"Task Progress"}, inplace=True)

In [9]:
df = df.astype({'Collaborators': 'string', 'Labels': 'string'})

df["Completed At"] = pd.to_datetime(df["Completed At"])
df["Due Date"] = pd.to_datetime(df["Due Date"])
df["Start Date"] = pd.to_datetime(df["Start Date"])
df["Created At"] = pd.to_datetime(df["Created At"])

In [10]:
df['Collaborators'] = df['Collaborators'].str.replace(';',',')

In [11]:
df['Created By'] = df['Created By'].replace({'Username_1':'Username_1_email',
                             'Username_2':'Username_2_email', 
                             'Username_3':'Username_3_email',
                             'Username_4':'Usernname_4_email', 
                             'Username_5':'Username_5_email',
                             'Username_6':'Username_6_email', 
                             'Username_7':'Username_7_email',
                             'Username_8':'Username_8_email'
                             })

In [12]:
df['Collaborators'] = df['Collaborators'].replace({'Username_1':'Username_1_email',
                             'Username_2':'Username_2_email', 
                             'Username_3':'Username_3_email',
                             'Username_4':'Usernname_4_email', 
                             'Username_5':'Username_5_email',
                             'Username_6':'Username_6_email', 
                             'Username_7':'Username_7_email',
                             'Username_8':'Username_8_email'})

In [13]:
df['Completed By'] = df['Completed By'].replace({'Username_1':'Username_1_email',
                             'Username_2':'Username_2_email', 
                             'Username_3':'Username_3_email',
                             'Username_4':'Usernname_4_email', 
                             'Username_5':'Username_5_email',
                             'Username_6':'Username_6_email', 
                             'Username_7':'Username_7_email',
                             'Username_8':'Username_8_email'})

In [14]:
ddf = pd.DataFrame(df["Collaborators"][df["Collaborators"].str.contains(',')].str.split(pat = ',', expand = True).replace({{'Username_1':'Username_1_email',
                             'Username_2':'Username_2_email', 
                             'Username_3':'Username_3_email',
                             'Username_4':'Usernname_4_email', 
                             'Username_5':'Username_5_email',
                             'Username_6':'Username_6_email', 
                             'Username_7':'Username_7_email',
                             'Username_8':'Username_8_email'}) )

df["Collaborators"][df["Collaborators"].str.contains(',')] = ddf[ddf.columns[0:]].apply(
    lambda x: ','.join(x.dropna().astype(str)),
    axis=1
)

In [15]:
df["Assignee"] = df["Collaborators"].str.split(',|/s', expand=True)[0]

In [16]:
pdf = df["Checklist Items"].str.split(pat = ';', expand = True)
pdf["Name"] = df.Name
pdf["Order3"] = df.Order3 + 1

In [17]:
size = pdf.shape[1]-2
a = pdf[[0,"Name","Order3"]][pdf[[0,"Name"]].notna().all(1)]

for i in range(1, size):
    b = pdf[[i,"Name","Order3"]][pdf[[i,"Name"]].notna().all(1)]
    b.rename(columns={i:0}, inplace=True)
    a = pd.concat([a,b],ignore_index=True)
    continue

In [18]:
a.rename(columns={0:"Name","Name":"Parent Task"}, inplace=True)

In [19]:
df["Parent Task"] = ''

In [20]:
df = pd.concat([df, a], ignore_index=True)

In [21]:
df.drop(columns=['Checklist Items'], inplace=True)

In [22]:
sec = df['Section/Column'].unique()
par = df['Parent Task'].unique()

In [23]:
df.sort_values("Order3", ignore_index=True, inplace=True)

In [24]:
xdf = df[["Created At", "Completed At", "Name", "Section/Column", "Assignee", "Start Date", "Due Date", "Notes", "Parent Task", "Task Progress", "Priority", "Collaborators", "Created By", "Late", "Completed By", "Completed Checklist Items", "Labels"]]

In [25]:
xdf.to_csv(Asana_format, index=False)